In [19]:
import matplotlib.pyplot as plt 
import numpy as np
import random
import keras
%matplotlib inline

In [20]:
# Load training text file
Data_train = np.loadtxt('training_data.txt', skiprows = 1)
X_train = Data_train[:, 1:]
y_train = Data_train[:, 0]
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (20000, 1000)
y_train shape: (20000,)


In [21]:
# Normalize training data
for i in range(len(X_train[0])):
    col = X_train[:, i]
    mean = np.mean(col)
    std = np.std(col)
    X_train[:, i] = (col - mean) / std

In [173]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras import regularizers

# Build sequential model
model = Sequential()

# Hidden layers
model.add(Dense(100, activation="sigmoid", input_shape=(1000,)))
model.add(Dropout(0.4))
model.add(Dense(100, activation="sigmoid"))
model.add(Dropout(0.4))
model.add(Dense(50, activation="sigmoid"))
model.add(Dropout(0.3))
# Output layer
model.add(Dense(1, activation="sigmoid"))
# Print a summary
model.summary()
# Compile
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_273 (Dense)            (None, 100)               100100    
_________________________________________________________________
dropout_198 (Dropout)        (None, 100)               0         
_________________________________________________________________
dense_274 (Dense)            (None, 100)               10100     
_________________________________________________________________
dropout_199 (Dropout)        (None, 100)               0         
_________________________________________________________________
dense_275 (Dense)            (None, 50)                5050      
_________________________________________________________________
dropout_200 (Dropout)        (None, 50)                0         
_________________________________________________________________
dense_276 (Dense)            (None, 1)                 51        
Total para

In [174]:
fit = model.fit(X_train, y_train, batch_size=300, epochs=15, validation_split=0.2, shuffle=True, verbose=1)

Train on 16000 samples, validate on 4000 samples
Epoch 1/15
16000/16000 [==============================] - 6s 350us/step - loss: 0.6999 - acc: 0.5344 - val_loss: 0.6197 - val_acc: 0.8075
Epoch 2/15
16000/16000 [==============================] - 1s 51us/step - loss: 0.5020 - acc: 0.7727 - val_loss: 0.3653 - val_acc: 0.8425
Epoch 3/15
16000/16000 [==============================] - 1s 51us/step - loss: 0.3651 - acc: 0.8472 - val_loss: 0.3522 - val_acc: 0.8515
Epoch 4/15
16000/16000 [==============================] - 1s 51us/step - loss: 0.3364 - acc: 0.8654 - val_loss: 0.3528 - val_acc: 0.8505
Epoch 5/15
16000/16000 [==============================] - 1s 54us/step - loss: 0.3257 - acc: 0.8684 - val_loss: 0.3534 - val_acc: 0.8515
Epoch 6/15
16000/16000 [==============================] - 1s 51us/step - loss: 0.3175 - acc: 0.8710 - val_loss: 0.3541 - val_acc: 0.8525
Epoch 7/15
16000/16000 [==============================] - 1s 51us/step - loss: 0.3097 - acc: 0.8732 - val_loss: 0.3543 - val_acc

In [94]:
## Printing the accuracy of our model, according to the loss function specified in model.compile above
score = model.evaluate(X_train, y_train, verbose=1)
print('Training score:', score[0])
print('Training accuracy:', score[1])

20000/20000 [==============================] - 1s 75us/step
Training score: 0.238491754043
Training accuracy: 0.91395


In [95]:
# Train on the entire training dataset
model = Sequential()
# Layers
model.add(Dense(500, activation="tanh", input_shape=(1000,)))
model.add(Dropout(0.5))
model.add(Dense(100, activation="sigmoid"))
model.add(Dropout(0.4))
model.add(Dense(1, activation="sigmoid"))
# Compile
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
# Fit
fit = model.fit(X_train, y_train, batch_size=256, epochs=5, shuffle=True, verbose=1)

# Make predictions
X_test = np.loadtxt('test_data.txt', skiprows = 1)

pred = model.predict(X_test)
super_threshold_indices = pred >= 0.5
pred.fill(0)
pred[super_threshold_indices] = 1

submission = [[i+1, int(pred[i])] for i in range(len(pred))]
submission.insert(0, ['Id','Prediction'])
with open('submission_neural_net.csv', 'w') as f:
    for line in submission:
        f.write(','.join(map(str, line)) + '\n')

Epoch 1/5
20000/20000 [==============================] - 5s 239us/step - loss: 0.4975 - acc: 0.7554
Epoch 2/5
20000/20000 [==============================] - 3s 147us/step - loss: 0.3345 - acc: 0.8587
Epoch 3/5
20000/20000 [==============================] - 3s 141us/step - loss: 0.3047 - acc: 0.8719
Epoch 4/5
20000/20000 [==============================] - 3s 143us/step - loss: 0.2840 - acc: 0.8825
Epoch 5/5
20000/20000 [==============================] - 3s 133us/step - loss: 0.2670 - acc: 0.8881


In [62]:
print(pred[0:5])

[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]]
